In [1]:
# Import the required packages
using JuMP
using CSV
using DataFrames
using PowerModels
using Ipopt
using Juniper

In [2]:
#Read in data
baseMVA = 10;

bus_data = CSV.read("bus_data.csv", DataFrame) |> Matrix;
(num_busses, var) = size(bus_data);
VMax = bus_data[:, 4];
VMin = bus_data[:, 7]; 
load_data = CSV.read("load_data.csv", DataFrame) |> Matrix;
(num_loads, var) = size(load_data);
branch_data = CSV.read("branch_data.csv", DataFrame) |> Matrix;
(num_branches, var) = size(branch_data)
G = CSV.read("G_mat.csv", DataFrame) |> Matrix;
B = CSV.read("B_mat.csv", DataFrame)|> Matrix;

#Expand the Pd data such that non load busses are given power demands of 0
Pd = load_data[:,5]*baseMVA
load_busses = load_data[:,2];
bus_idx = bus_data[:,2]; 
Pd_mod = zeros(1, num_busses);
for i = 1:num_busses
    idx = findall(isequal(bus_idx[i]), load_busses)
    if !isempty(idx)
        Pd_mod[i] = Pd[idx[1]]
    else
        Pd_mod[i] = 0;
    end
end
Pd = Pd_mod

Qd = load_data[:,4]*baseMVA
Qd_mod = zeros(1, num_busses);
for i = 1:num_busses
    idx = findall(isequal(bus_idx[i]), load_busses)
    if !isempty(idx)
        Qd_mod[i] = Qd[idx[1]]
    else
        Qd_mod[i] = 0;
    end
end
Qd = Qd_mod

1×18 Matrix{Float64}:
 0.0  0.12  0.25  0.93  2.26  0.5  0.12  …  0.5  0.31  0.62  0.12  0.0  0.0

In [ ]:
optimizer = Juniper.Optimizer
nl_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)

model = Model(optimizer_with_attributes(optimizer, "nl_solver"=>nl_solver))

@variable(model, V[1:num_busses] >= 0)
@variable(model, theta[1:num_busses])
@variable(model, PG[1:num_busses] >= 0)
@variable(model, QG[1:num_busses] >= 0)
@variable(model, y[1:num_busses] >= 0)
@variable(model, x[1:num_busses], Bin)
@variable(model, Ploss[1:num_busses] >= 0)
@variable(model, Qloss[1:num_busses] >= 0)
@variable(model, I[1:num_busses, 1:num_busses] >= 0)
@variable(model, VP >= 0)

M = 10000;

@constraint(model, [i in 1:num_busses], y[i] <= M*x[i])
@NLconstraint(model, [i in 1:num_busses], PG[i] - Pd[i] + Ploss[i] == sum((V[i]*V[j])*(G[i,j]*cos(theta[i]-theta[j])+B[i,j]*sin(theta[i]-theta[j])) for j in 1:num_busses))
@NLconstraint(model, [i in 1:num_busses], QG[i] - Qd[i] == sum((V[i]*V[j])*(G[i,j]*sin(theta[i]-theta[j])-B[i,j]*cos(theta[i]-theta[j])) for j in 1:num_busses))
#This case has no flow limits
#@NLconstraint(model, [i in 1:num_busses, j in 1:num_busses], I[i,j] == sqrt(G[i,j]^2+B[i,j]^2)*sqrt(V[i]^2+V[j]^2-2*V[i]*V[j]*cos(theta[j]-theta[i])))
@constraint(model, [i in 1:num_busses], (PG[i]^2+QG[i]^2)^(1/2) <= y[i])
#@constraint(model, [i in 1:num_busses], QG[i] <= y[i])
#This case has no flow limits
#@constraint(model, [i in 1:num_busses, j in 1:num_busses] I[i,j] <= Imax[i,j])
@constraint(model, [i in 1:num_busses-1], VMin[i] <= V[i] <= VMax[i])
@constraint(model, V[num_busses] == 0)
@constraint(model, theta[num_busses] == 0)
@constraint(model, VP == sum(V[i]*Pd[i] for i in 1:num_busses))
@constraint(model, [i in 1:num_busses],  -90 <= theta[i] <= 90 )


c_fix = 10;
c_var = 1;
beta = .5;
gamma = 50;

@objective(model, Min, sum(c_fix*x[i]+c_var*y[i] for i in 1:num_busses) +
                        beta*VP +
                        gamma*sum(Ploss[i] for i in 1:num_busses))

optimize!(model)


nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("print_level") => 0])
feasibility_pump  : false
log_levels        : [:Options, :Table, :Info]

#Variables: 469
#IntBinVar: 18
Obj Sense: Min

Start values are not feasible.
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 0.22699999809265137
Relaxation Obj: 279.6923481279462

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time   Restarts  GainGap  
    2       2                 -                         279.69              -     5.3       2         -     
    3       3                 -                         289.66              -     5.5       -       98.6%   
    4       4                 -                         289.78              -     5.6       -       99.5%   
    5       5                 -                         290.0               -     5.8       -       100.0%

┌ Warning: Only almost solved
└ @ Juniper C:\Users\riley\.julia\packages\Juniper\HBPrQ\src\BnBTree.jl:116


   26       7                 -                         299.95              -     9.1       -       95.5%   
   27       8                 -                         299.95              -     9.2       -       95.2%   
   28       9                 -                         299.95              -     9.5       -       94.5%   
   29       10                -                         299.95              -     9.6       -       94.4%   
   30       11                -                         299.95              -     9.7       -       99.3%   
   31       12                -                         299.95              -     9.9       -       99.2%   
   32       13                -                         299.95              -     10.0      -       99.4%   
   33       6                 -                         300.0               -     10.2      -       99.4%   
   34       14                -                         300.0               -     10.3      -       99.8%   
   35       7      

In [18]:
optimizer = Juniper.Optimizer
nl_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)

model = Model(optimizer_with_attributes(optimizer, "nl_solver"=>nl_solver))

@variable(model, V[1:num_busses] >= 0)
@variable(model, theta[1:num_busses])
@variable(model, PG[1:num_busses] >= 0)
@variable(model, QG[1:num_busses] >= 0)
@variable(model, y[1:num_busses] >= 0)
@variable(model, x[1:num_busses], Bin)
@variable(model, Ploss[1:num_busses] >= 0)
@variable(model, Qloss[1:num_busses] >= 0)
@variable(model, I[1:num_busses, 1:num_busses] >= 0)
@variable(model, VP >= 0)

M = 10000;

#In bus data, a 3 indicates the slack bus, the connection to the grid
slack_idx = findall(isequal(3), bus_data[:,3])
non_slack_num = filter(x->x!=slack_idx, 1:num_busses)

@constraint(model, [i in 1:num_busses], y[i] <= M*x[i])
@NLconstraint(model, [i in 1:num_busses], PG[i] - Pd[i] + Ploss[i] == sum((V[i]*V[j])*(G[i,j]*cos(theta[i]-theta[j])+B[i,j]*sin(theta[i]-theta[j])) for j in 1:num_busses))
@constraint(model, [i in 1:num_busses], Ploss[i] <= Pd[i])
@NLconstraint(model, [i in 1:num_busses], QG[i] - Qd[i] == sum((V[i]*V[j])*(G[i,j]*sin(theta[i]-theta[j])-B[i,j]*cos(theta[i]-theta[j])) for j in 1:num_busses))
#This case has no flow limits
#@NLconstraint(model, [i in 1:num_busses, j in 1:num_busses], I[i,j] == sqrt(G[i,j]^2+B[i,j]^2)*sqrt(V[i]^2+V[j]^2-2*V[i]*V[j]*cos(theta[j]-theta[i])))
#When using capacity constraint sqrt(Pg^2+Qg^2) <= y, becomes infeasible
#@constraint(model, [i in 1:num_busses], PG[i]+QG[i] <= y[i])
@NLconstraint(model, [i in 1:num_busses], (PG[i]^2+QG[i]^2)^(1/2) <= y[i])
#This case has no flow limits
#@constraint(model, [i in 1:num_busses, j in 1:num_busses] I[i,j] <= Imax[i,j])
@constraint(model, [i in non_slack_num], VMin[i] <= V[i] <= VMax[i])
#@constraint(model, V[1] == 0)
#@constraint(model, theta[1] == 0)
@constraint(model, VP == sum(V[i]*Pd[i] for i in 1:num_busses))
@constraint(model, [i in 1:num_busses],  -pi/2 <= theta[i] <= pi/2 )

#Varying these parameters changes output behavior
#When gamma < 1, tends to just shed all load, bad!
#For gamma = 1, and slightly above sheds some load, for gamma well above 1, like 2, 
#Serves all load. Interesting! Hyper parameter tuning 
c_fix = 1;
c_var = 1;
beta = .5;
gamma = 5;

@objective(model, Min, sum(c_fix*x[i]+c_var*y[i] for i in 1:num_busses) +
                        beta*VP +
                        gamma*sum(Ploss[i] for i in 1:num_busses))

optimize!(model)

nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("print_level") => 0])
feasibility_pump  : false
log_levels        : [:Options, :Table, :Info]

#Variables: 469
#IntBinVar: 18
Obj Sense: Min

Start values are not feasible.
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 0.2760000228881836
Relaxation Obj: 18.92324171367204

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time   Restarts  GainGap  
    2       2                 -                         18.92               -     1.9       0         -     
    3       3                 -                         19.07               -     2.1       -       99.7%   
    4       4                 -                         19.13               -     2.2       -       99.9%   
    5       5                 -                         19.27               -     2.4       -       100.0% 

In [20]:
JuMP.value.(y)

18-element Vector{Float64}:
 3.2314200349723055
 4.099071832945378e-9
 4.611635652735249e-9
 4.719400454281647e-9
 7.356948031554203
 4.596967429372778e-9
 6.8010796462453045e-9
 8.294223775151243e-9
 4.7816606301092175e-9
 5.480504637196012e-9
 5.3532968089440164e-9
 5.958562568121913e-9
 3.273335591111857
 4.6277844988998375e-9
 6.832171906001012e-9
 6.9435136957297195e-9
 5.231592834941418e-9
 5.320087338991099e-9

In [95]:
V = JuMP.value.(V)
theta = JuMP.value.(theta)
i = 17
println(sum((V[i]*V[j])*(G[i,j]*cos(theta[i]-theta[j])+B[i,j]*sin(theta[i]-theta[j])) for j in 1:num_busses))
println(sum(JuMP.value.(PG)))
println(sum(Pd))


34.10291728708915
41.33766404557758
7.6
